In [1]:
%matplotlib
%matplotlib notebook

import numpy as np
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt

header = ['MouthHook', 'LeftMHhook', 'RightMHhook',
                   'LeftDorsalOrgan', 'RightDorsalOrgan',
                   'CenterBolwigOrgan']

Using matplotlib backend: MacOSX


In [2]:
## Old Data
# rocFile = os.path.join('ROC_20170317_7c2.pkl')
rocFile = os.path.join('ROC_20170317_7c0.pkl')
# rocFile = os.path.join('ROC_20170317_7c1.pkl')

## New Data
# rocFile = os.path.join('ROC_20180417_7c2.pkl')
# rocFile = os.path.join('ROC_20180417_7c1.pkl')
# rocFile = os.path.join('ROC_20180417_7c0.pkl')

with open(rocFile, 'r') as rf:
    rocDatGrpAll = pickle.load(rf)

In [3]:
vote_threshold = np.array(rocDatGrpAll[0]['MouthHook'][25].loc[:, 'vote_threshold'].values, dtype=np.float16)
recall_all = {}
dist_all = {}
for grp in rocDatGrpAll:
    for bp in grp:
        recall_all[bp] = {}
        dist_all[bp] = {}
        for thresh in grp[bp]:
            recall_all[bp][thresh] = pd.DataFrame([])
            dist_all[bp][thresh] = pd.DataFrame([])            

for grp in rocDatGrpAll:
    for bp in grp:
        for thresh in grp[bp]:
            recall_all[bp][thresh] = pd.concat([recall_all[bp][thresh], grp[bp][thresh].loc[:, 'recall']], axis=1)
            dist_all[bp][thresh] = pd.concat([recall_all[bp][thresh], grp[bp][thresh].loc[:, 'recall']], axis=1)            
            

In [4]:
grp[bp][thresh].columns

Index([u'true_positive', u'false_positive', u'true_negative',
       u'false_negative', u'positive', u'negative', u'true_positive_rate',
       u'false_positive_rate', u'total_negative', u'total_positive',
       u'true_negative_rate', u'false_negative_rate', u'precision', u'recall',
       u'vote_threshold'],
      dtype='object')

In [6]:
row, col = (0, 0)
fig, ax = plt.subplots(2, 3, figsize=(18, 15), dpi=90)
for bp in header:
    for thresh in recall_all[bp]:
        recall_all_mat = np.array(recall_all[bp][thresh], dtype=np.float32)
        recall_mean = np.mean(recall_all_mat, axis=1)
        recall_std = np.std(recall_all_mat, axis=1)
        ax[row, col].plot(vote_threshold, recall_mean, lw=3, label=int(thresh))
        ax[row, col].fill_between(vote_threshold, recall_mean-recall_std, recall_mean+recall_std, alpha=0.1, lw=0)

    ax[row, col].set_xlabel('Vote Threshold', fontsize=18)
    ax[row, col].set_ylabel('Recall', fontsize=18)
    ax[row, col].set_title(bp, fontsize=20)
    ax[row, col].set_aspect('equal', adjustable='box')
    ax[row, col].set_xlim([-0.05, 1.05])
    ax[row, col].set_ylim([-0.05, 1.05])
    ax[row, col].spines['right'].set_visible(False)
    ax[row, col].spines['top'].set_visible(False)
    ax[row, col].legend(loc = 'upper right', frameon=False, fontsize=14, title='Distance')
    for label in (ax[row, col].get_xticklabels() + ax[row, col].get_yticklabels()):
        label.set_fontsize(18)

    col += 1
    if col > 2:
        row += 1
        col = 0